This notebook is the gridsearch done with previous data. The previous data was processed by XCMS WITH the PHD data, using the xcms script optimized previously. This optimization of XCMS parameters was done only with data from the PHD. This could be the reason behind the 0.0 score obtained at the end. So, to verify this, the next notebook had the previous data together with the PHD data during IPO optimization

# Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.model_selection import PredefinedSplit


# Functions

In [2]:
# creates the feature name with the mz and rt

def feature_name_creation(xcms_file_path):
    table = pd.read_csv(xcms_file_path, index_col=[0]) 
    
    # no need for decimal on m/z (low resolution) and only one decimal for rt
    table.mz = table.mz.round(0).astype(int)
    table.rt = table.rt.round(1)

    # creating the feature name: mz_rt
    features = table["mz"].astype(str) + "_" + table["rt"].astype(str)
    table.insert(0, 'features', features) # first column
    
    # drop as we don't know how many columns the table will have. Drop the known ones. 
    # There should only be the 'features' column and the samples
    
    table_clean = table.drop(['isotopes', 'adduct','pcgroup'], axis=1) #'npeaks','NEG_GROUP', 'POS_GROUP',
    
    return table_clean

In [3]:
# rounds the mz and rt columns along with its min and max

def rounder(dataframe):
    table = dataframe 
    
    table.mz = table.mz.round(0).astype(int)
    table.mzmin = table.mzmin.round(0).astype(int)
    table.mzmax = table.mzmax.round(0).astype(int)
    
    table.rt = table.rt.round(1)
    table.rtmin = table.rtmin.round(1)
    table.rtmax = table.rtmax.round(1)

    
    return table

# Data Prep Pipeline

`Train` and `Val` sets were processed separately on `xcms` - excludes the possibility of data leakage 
But, when processing is separated, the features can be slightly different. The compounds are almost the same, but due to processing steps, there can be shifts on the decimals of `mz` or `rt`. 
For this reason, creating the feature name concatenating `mz_rt` on train and val might not produce the same features, and machine learning training is not possible with that. 

Errors observed in this case are related to the fact that features observed in train were not present in validation and vice versa or the order of the features were different in both datasets. This 'pipeline' fixes this issue.

**Steps:**

1. Creates the name for the features on `Train` set - this is the set used as reference. Whatever features where observed here, should appear on `Val`. The name is created concatenating `mz` and `rt` columns (`mz_rt`)
2. Creates a correspondance between the feature on `Train` and `Val` set, giving val set the same column names as the train, when the feature is present 
    1. round `mz` and `rt` from `Val` and `Train` 
    2. for each `mz` in `Val`, search for a range on `mzmin` and `mzmax` on train that fits. The `mz_val` need to be between `mzmin_train` and `mzmax_train` 
    3. If a match is found, for each `rt`,`rtmin` and `rtmax` on `Val` search for a range on `rtmin` and `rtmax` on `Train` that fits. The `rt` values need to be between `rtmin_train` and `rtmax_train`. The `rtmin` and `rtmax` from `Val` are used in this case because ocasionally, the range on `Val` or train is too big (big difference in `rt` between samples)
    4. if a match, take the feature name from `Train` and apply on the match
    
**With the features names created:**

3. Features on `Train` and `Val` are ordered 
4. Duplicates are deleted based on the `npeaks` columnn
5. Features that were observed in `Val` but no correspondence was found in `Train` have names filled with `nan`. These are deleted.
4. Features that are on `Train`and were not found in `Val` are added to `Val` and filled with zero (no presence of that feature)
 
 
**To fix: **
 The code for the feature correspondence is not optimized. 
 - After the match with `mz`, the loop searches on the whole dataset for a match in `rt`. This takes more computation, unecessary. 
 - If there is a match of two features, the last one is kept. Could keep both, filter later? 
 


## Feature reference creation - train set

In [4]:
# train is loaded using the function to create the feature names - feature names are created using mz and rt.  
maytenus_train = feature_name_creation('5. Gridsearch/maytenus_train_processing.csv').reset_index(drop=True)

In [5]:
maytenus_train.head()

,features,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,NEG_GROUP,POS_GROUP,...,IL9_3,IL96_1,IL96_2,IL96_3,IL97_1,IL97_2,IL97_3,IL99_1,IL99_2,IL99_3
0,118_574.6,118,116.844856,117.830395,574.6,446.674,575.801,117,45,56,...,5.908828e+07,5.899383e+07,5.588464e+07,5.612618e+07,4.912388e+07,4.923401e+07,4.929297e+07,4.667222e+07,4.616037e+07,4.839906e+07
1,118_574.2,118,117.857061,118.844673,574.2,572.723,575.897,307,47,69,...,6.326028e+07,6.241781e+07,5.908452e+07,6.031819e+07,6.075755e+07,5.855685e+07,5.976457e+07,5.913456e+07,5.768406e+07,6.095912e+07
2,133_63.3,133,132.418292,132.844587,63.3,58.987,68.379,453,101,109,...,3.002733e+08,4.824321e+08,4.839829e+08,4.671959e+08,3.006155e+08,2.944828e+08,2.758049e+08,2.326754e+08,2.436494e+08,2.364542e+08
3,133_63.2,133,132.844790,133.330327,63.2,58.987,68.386,8638,378,384,...,2.960507e+08,4.595602e+08,4.676176e+08,4.621277e+08,3.016531e+08,3.243751e+08,2.951865e+08,2.341830e+08,2.399732e+08,2.361439e+08
4,163_360.9,163,162.679173,163.251287,360.9,358.839,363.122,132,51,0,...,1.187537e+07,1.161814e+07,1.151374e+07,1.001517e+07,7.565125e+06,8.315961e+06,7.388538e+06,7.563753e+06,7.313061e+06,7.200719e+06


## Loading validation val set

In [6]:
# val will be loaded using regular read_csv - the names of the features will come based on comparison
maytenus_val = pd.read_csv('5. Gridsearch/maytenus_validation_processing.csv',index_col=[0]).reset_index(drop=True).drop(['isotopes', 'adduct','pcgroup'], axis=1) #'npeaks','NEG_GROUP', 'POS_GROUP',

## Rounding mz and rt

In [7]:
# rouding all mz and all rt
maytenus_val = rounder(maytenus_val)
maytenus_train = rounder(maytenus_train)

In [8]:
display(maytenus_val.iloc[:,0:7].head())
display(maytenus_train.iloc[:,0:7].head())

,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks
0,117,117,117,112.2,111.3,113.7,20
1,118,117,118,574.2,573.5,575.0,13
2,118,118,119,573.8,573.1,574.6,53
3,133,133,133,60.3,57.1,65.7,2246
4,163,163,163,348.1,344.6,352.2,54


,features,mz,mzmin,mzmax,rt,rtmin,rtmax
0,118_574.6,118,117,118,574.6,446.7,575.8
1,118_574.2,118,118,119,574.2,572.7,575.9
2,133_63.3,133,132,133,63.3,59.0,68.4
3,133_63.2,133,133,133,63.2,59.0,68.4
4,163_360.9,163,163,163,360.9,358.8,363.1


In [9]:
display(maytenus_val)
display(maytenus_train)

,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,NEG_GROUP,POS_GROUP,AQ15_1,...,IL88_3,IL89_1,IL89_2,IL89_3,IL90_1,IL90_2,IL90_3,IL93_1,IL93_2,IL93_3
0,117,117,117,112.2,111.3,113.7,20,12,0,1.507722e+07,...,1.461273e+07,1.434329e+07,1.436273e+07,1.391806e+07,9.000562e+06,9.721818e+06,9.042574e+06,1.257923e+07,1.336079e+07,1.316295e+07
1,118,117,118,574.2,573.5,575.0,13,3,10,4.016225e+07,...,5.592482e+07,5.365049e+07,5.376085e+07,5.381437e+07,6.404797e+07,6.219705e+07,6.203724e+07,5.224481e+07,5.518170e+07,5.498099e+07
2,118,118,119,573.8,573.1,574.6,53,7,15,4.153075e+07,...,5.857564e+07,6.322207e+07,6.139799e+07,6.090669e+07,6.834088e+07,6.645232e+07,6.623445e+07,5.886957e+07,5.755579e+07,6.050415e+07
3,133,133,133,60.3,57.1,65.7,2246,96,96,4.548042e+08,...,4.815065e+08,3.174242e+08,3.120228e+08,3.069786e+08,3.021286e+08,2.978952e+08,2.971043e+08,3.217211e+08,3.153699e+08,3.231264e+08
4,163,163,163,348.1,344.6,352.2,54,12,0,9.214956e+06,...,9.397630e+06,9.037046e+06,7.827418e+06,8.148794e+06,1.124304e+07,1.112709e+07,1.236415e+07,9.509602e+06,8.799963e+06,8.076247e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,741,741,742,284.4,282.3,287.8,94,0,94,1.854838e+07,...,9.428788e+07,8.643290e+07,7.781434e+07,7.667890e+07,8.215795e+07,8.672896e+07,8.572576e+07,6.567109e+07,7.106116e+07,6.721195e+07
94,755,754,755,268.7,266.9,271.1,42,0,42,1.871405e+07,...,8.031803e+07,7.000626e+07,7.291199e+07,5.532321e+07,6.726124e+07,6.271704e+07,7.129912e+07,4.220301e+07,4.313111e+07,4.445439e+07
95,755,755,756,268.6,264.8,271.5,999,0,96,1.937589e+07,...,8.416279e+07,7.747889e+07,7.838830e+07,7.654461e+07,6.726851e+07,8.314858e+07,7.156967e+07,4.917694e+07,5.297594e+07,4.751466e+07
96,756,756,757,268.7,265.2,271.6,193,0,96,1.762526e+07,...,7.710153e+07,5.676400e+07,5.879134e+07,6.519987e+07,6.654037e+07,6.715041e+07,6.434036e+07,4.980773e+07,5.314412e+07,4.697505e+07


,features,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,NEG_GROUP,POS_GROUP,...,IL9_3,IL96_1,IL96_2,IL96_3,IL97_1,IL97_2,IL97_3,IL99_1,IL99_2,IL99_3
0,118_574.6,118,117,118,574.6,446.7,575.8,117,45,56,...,5.908828e+07,5.899383e+07,5.588464e+07,5.612618e+07,4.912388e+07,4.923401e+07,4.929297e+07,4.667222e+07,4.616037e+07,4.839906e+07
1,118_574.2,118,118,119,574.2,572.7,575.9,307,47,69,...,6.326028e+07,6.241781e+07,5.908452e+07,6.031819e+07,6.075755e+07,5.855685e+07,5.976457e+07,5.913456e+07,5.768406e+07,6.095912e+07
2,133_63.3,133,132,133,63.3,59.0,68.4,453,101,109,...,3.002733e+08,4.824321e+08,4.839829e+08,4.671959e+08,3.006155e+08,2.944828e+08,2.758049e+08,2.326754e+08,2.436494e+08,2.364542e+08
3,133_63.2,133,133,133,63.2,59.0,68.4,8638,378,384,...,2.960507e+08,4.595602e+08,4.676176e+08,4.621277e+08,3.016531e+08,3.243751e+08,2.951865e+08,2.341830e+08,2.399732e+08,2.361439e+08
4,163_360.9,163,163,163,360.9,358.8,363.1,132,51,0,...,1.187537e+07,1.161814e+07,1.151374e+07,1.001517e+07,7.565125e+06,8.315961e+06,7.388538e+06,7.563753e+06,7.313061e+06,7.200719e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,741_302.0,741,740,741,302.0,264.8,311.0,350,5,345,...,8.452444e+07,7.295674e+07,7.044123e+07,6.966945e+07,9.884958e+07,1.105645e+08,1.005150e+08,1.062612e+08,9.917727e+07,9.797857e+07
89,755_286.3,755,754,755,286.3,248.8,312.9,1032,5,351,...,8.833914e+07,5.279847e+07,5.557496e+07,4.565162e+07,1.261003e+08,1.379112e+08,1.216284e+08,7.109022e+07,8.046582e+07,8.627527e+07
90,756_286.7,756,755,756,286.7,248.8,312.9,3392,5,351,...,1.016305e+08,5.368691e+07,5.566762e+07,5.216342e+07,1.550416e+08,1.187256e+08,1.537047e+08,9.148052e+07,9.322778e+07,9.990198e+07
91,757_286.5,757,756,757,286.5,256.7,312.9,157,1,153,...,8.192823e+07,4.605462e+07,4.654267e+07,4.420568e+07,5.858005e+07,6.576977e+07,6.590599e+07,7.515455e+07,7.053095e+07,4.315254e+07


## Feature creation and correspondance on val set - create a function of this

In [10]:
# creating the column
maytenus_val['features'] = np.nan

In [11]:
# loop over maytenus_val items. 
# Each mz will be tested against all mzmin and mzmax range from train. 
# if in range, test for rt.
# if in range, use the same feature name from train

maytenus_val = maytenus_val.sort_values('npeaks', ascending=False,ignore_index=True)
maytenus_train_ref = maytenus_train.sort_values('npeaks', ascending=False,ignore_index=True)

for i in range(len(maytenus_val)):
    for j in range(len(maytenus_train_ref)):


        if ((maytenus_val.loc[i,'mz'] <= maytenus_train_ref.loc[j,'mzmax']) 
              & (maytenus_val.loc[i,'mz'] >= maytenus_train_ref.loc[j,'mzmin'])):
            
            #maybe subset maytenus train and then perform things on the subset? 
            
            if (
                ((maytenus_val.loc[i,'rt'] <= maytenus_train_ref.loc[j,'rtmax']) 
                  & (maytenus_val.loc[i,'rt'] >= maytenus_train_ref.loc[j,'rtmin'])) or
            
               ((maytenus_val.loc[i,'rtmin'] <= maytenus_train_ref.loc[j,'rtmax']) 
                  & (maytenus_val.loc[i,'rtmin'] >= maytenus_train_ref.loc[j,'rtmin'])) or
                
               ((maytenus_val.loc[i,'rtmax'] <= maytenus_train_ref.loc[j,'rtmax']) 
                & (maytenus_val.loc[i,'rtmax'] >= maytenus_train_ref.loc[j,'rtmin']))
            ):
                
                maytenus_val.loc[i,'features'] = maytenus_train_ref.loc[j,'features']
            break

In [12]:
maytenus_val

,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,NEG_GROUP,POS_GROUP,AQ15_1,...,IL89_1,IL89_2,IL89_3,IL90_1,IL90_2,IL90_3,IL93_1,IL93_2,IL93_3,features
0,191,191,192,89.0,50.3,92.2,2248,96,93,1.888418e+08,...,2.563923e+08,2.509333e+08,2.487752e+08,2.339792e+08,2.309708e+08,2.304622e+08,2.391413e+08,2.371037e+08,2.361392e+08,191_102.3
1,133,133,133,60.3,57.1,65.7,2246,96,96,4.548042e+08,...,3.174242e+08,3.120228e+08,3.069786e+08,3.021286e+08,2.978952e+08,2.971043e+08,3.217211e+08,3.153699e+08,3.231264e+08,133_63.2
2,289,288,289,254.4,220.8,259.4,1759,36,96,9.599036e+07,...,1.920411e+08,1.953578e+08,1.917544e+08,2.147209e+08,2.108924e+08,2.091615e+08,1.348852e+08,1.373190e+08,1.387906e+08,289_272.2
3,561,561,562,265.1,227.6,270.1,1258,28,78,9.315947e+07,...,1.333482e+08,1.354178e+08,1.390231e+08,1.424696e+08,1.403660e+08,1.364839e+08,1.340487e+08,1.327994e+08,1.432326e+08,561_282.7
4,739,739,740,284.5,282.3,288.3,1155,0,96,2.747926e+07,...,2.072239e+08,2.044367e+08,1.726183e+08,1.804404e+08,2.048974e+08,1.819017e+08,1.652030e+08,1.686066e+08,1.687056e+08,739_302.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,654,654,655,266.5,263.0,269.7,18,15,0,2.193694e+07,...,1.780166e+07,1.746834e+07,1.990690e+07,1.921297e+07,1.796969e+07,1.756218e+07,2.042788e+07,1.980194e+07,2.076722e+07,NaN
94,118,117,118,574.2,573.5,575.0,13,3,10,4.016225e+07,...,5.365049e+07,5.376085e+07,5.381437e+07,6.404797e+07,6.219705e+07,6.203724e+07,5.224481e+07,5.518170e+07,5.498099e+07,118_574.2
95,424,424,424,410.7,409.0,412.4,12,12,0,1.207037e+07,...,1.140325e+07,1.162769e+07,1.130129e+07,1.156243e+07,9.358604e+06,8.610475e+06,9.908140e+06,1.160593e+07,8.717365e+06,NaN
96,657,656,657,262.6,260.5,264.7,11,11,0,2.096774e+07,...,1.791879e+07,1.684114e+07,1.941615e+07,1.911588e+07,1.719130e+07,1.739270e+07,1.952732e+07,1.971833e+07,1.862377e+07,NaN


In [13]:
# the process can create duplicates, so removing them is necessary
# the removal is based on the npeaks column. The feature with more npeaks, is kept.
maytenus_val = maytenus_val.sort_values('npeaks', ascending=False).drop_duplicates('features').sort_index()

# dropping unnecessary columns
maytenus_val = maytenus_val.drop(['mz', 'mzmin', 'mzmax', 'rt', 
                                  'rtmin', 'rtmax', 'npeaks','NEG_GROUP', 'POS_GROUP'], axis=1)

# removing the duplicates that might arise with the train is also necessary
# drop possible duplicates for train as well
maytenus_train_ref = maytenus_train_ref.sort_values('npeaks', ascending=False).drop_duplicates('features').sort_index()

# dropping unnecessary  columns
maytenus_train_ref = maytenus_train_ref.drop(['mz', 'mzmin', 'mzmax', 'rt', 
                                      'rtmin', 'rtmax', 'npeaks','NEG_GROUP', 'POS_GROUP'], axis=1)

# val set might have some feature that don't fit in any range - their feature names will be nan, so need to remove
# train might have some features that wont appear in the val. So, create them in val and set them to zero. 
# first, set index on both to be the features, so its possible to do that.
maytenus_train_ref= maytenus_train_ref.set_index('features')
maytenus_val = maytenus_val.dropna().set_index('features') # dropping na and making feature as index

# set method to get the set of index values that are unique 
# subtracting the sets to get the different indexes. 
# concat method to concatenate train and val
# filling the missing values on the concatenation with 0 using the fillna method.

unique_indexes = list(set(maytenus_train_ref.index) - set(maytenus_val.index))
maytenus_val = pd.concat([maytenus_val, pd.DataFrame(index=unique_indexes, columns=maytenus_val.columns)], sort=True).fillna(0)

# order both val and train features equally
# sort the features - the model needs them at the same sequence
maytenus_train_grid = maytenus_train_ref.reset_index().sort_values(by='features')
maytenus_val = maytenus_val.reset_index().sort_values(by='index')



In [14]:
maytenus_val

,index,AQ15_1,AQ15_2,AQ15_3,AQ24_1,AQ24_2,AQ24_3,AQ29_1,AQ29_2,AQ29_3,...,ML43_3,ML44_1,ML44_2,ML44_3,ML46_1,ML46_2,ML46_3,ML49_1,ML49_2,ML49_3
41,118_574.2,4.153075e+07,4.128833e+07,4.127786e+07,5.993508e+07,5.888217e+07,6.205178e+07,6.226222e+07,6.362410e+07,6.167428e+07,...,6.069511e+07,6.183458e+07,5.985650e+07,5.639033e+07,5.428191e+07,4.985583e+07,4.867924e+07,5.155424e+07,5.018745e+07,5.006901e+07
59,118_574.6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,133_63.2,4.548042e+08,4.467540e+08,4.587334e+08,2.692450e+08,2.620074e+08,2.490009e+08,2.720095e+08,2.681214e+08,2.814799e+08,...,4.178706e+08,1.221708e+08,1.207355e+08,1.183927e+08,4.058158e+08,4.155007e+08,4.077516e+08,4.329363e+08,4.456691e+08,4.328093e+08
49,133_63.3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
63,163_360.9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,741_302.0,1.854838e+07,2.000525e+07,1.990883e+07,1.691154e+07,1.758071e+07,1.848960e+07,1.611531e+07,1.789688e+07,1.758252e+07,...,1.656812e+07,1.689603e+07,1.595879e+07,1.599404e+07,1.634736e+07,1.584006e+07,1.611701e+07,1.688472e+07,1.578628e+07,1.783517e+07
72,755_286.3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,756_286.7,1.937589e+07,1.996009e+07,1.937854e+07,1.927812e+07,1.803453e+07,1.568489e+07,1.638128e+07,1.687270e+07,1.556628e+07,...,1.579348e+07,1.633861e+07,1.521654e+07,1.360705e+07,1.633646e+07,1.695084e+07,1.616958e+07,1.729669e+07,1.721778e+07,1.677490e+07
44,757_286.5,1.683946e+07,1.733512e+07,1.604495e+07,1.474818e+07,1.498274e+07,1.323982e+07,1.568520e+07,1.555287e+07,1.619717e+07,...,2.041815e+07,1.683738e+07,1.578859e+07,1.429213e+07,1.935499e+07,2.012559e+07,2.025872e+07,2.011256e+07,2.034606e+07,1.941281e+07


## Bring the class data column

In [15]:
# load
classes_train = pd.read_csv('classes_train_maytenus.csv', index_col=[0])
classes_val = pd.read_csv('classes_val_maytenus.csv', index_col=[0])

# unite
maytenus_train_grid = maytenus_train_grid.set_index('features').T.join(classes_train)
display(maytenus_train_grid.head())

maytenus_val = maytenus_val.set_index('index').T.join(classes_val)
display(maytenus_val.head())

,118_574.2,118_574.6,133_63.2,133_63.3,163_360.9,164_351.6,165_349.9,181_45.3,181_45.6,191_101.4,...,707_241.4,707_241.5,709_241.9,739_302.0,741_302.0,755_286.3,756_286.7,757_286.5,758_286.7,class
AQ1_1,6.650201e+07,5.075549e+07,3.521291e+08,3.616098e+08,9.044279e+06,1.457090e+07,1.596639e+07,7.361370e+07,7.336789e+07,2.630690e+08,...,1.325826e+07,1.420363e+07,1.399070e+07,1.846844e+07,1.801901e+07,1.324304e+07,1.624732e+07,1.707240e+07,1.595537e+07,0
AQ1_2,6.424319e+07,4.872172e+07,3.544615e+08,3.624862e+08,9.299879e+06,1.403339e+07,1.621949e+07,7.310785e+07,7.265559e+07,2.691049e+08,...,1.111815e+07,1.164364e+07,1.380097e+07,1.810845e+07,1.499541e+07,1.578869e+07,1.707935e+07,1.649287e+07,1.598885e+07,0
AQ1_3,6.366389e+07,4.700509e+07,3.534877e+08,3.670777e+08,8.167617e+06,1.478507e+07,1.532995e+07,7.485959e+07,7.319497e+07,2.718233e+08,...,1.116593e+07,1.223106e+07,1.255418e+07,1.847344e+07,1.644607e+07,1.386874e+07,1.538919e+07,1.589368e+07,1.506255e+07,0
AQ10_1,6.530954e+07,6.366778e+07,3.912875e+08,4.046616e+08,1.115645e+07,1.560228e+07,2.241762e+07,7.216161e+07,7.069488e+07,2.017449e+08,...,1.166157e+07,1.338020e+07,1.317420e+07,2.008873e+07,1.695123e+07,1.627024e+07,1.829186e+07,1.562728e+07,1.487267e+07,0
AQ10_2,6.516760e+07,6.251803e+07,3.872449e+08,3.971896e+08,1.262390e+07,1.647965e+07,2.316794e+07,7.510947e+07,7.226401e+07,1.947154e+08,...,1.184230e+07,1.338872e+07,1.325377e+07,1.822503e+07,1.791410e+07,1.615965e+07,1.810262e+07,1.770847e+07,1.659001e+07,0


,118_574.2,118_574.6,133_63.2,133_63.3,163_360.9,164_351.6,165_349.9,181_45.3,181_45.6,191_101.4,...,707_241.4,707_241.5,709_241.9,739_302.0,741_302.0,755_286.3,756_286.7,757_286.5,758_286.7,class
AQ15_1,4.153075e+07,0.0,4.548042e+08,0.0,0.0,0.0,0.0,0.0,6.644129e+07,0.0,...,0.0,0.0,0.0,2.747926e+07,1.854838e+07,0.0,1.937589e+07,1.683946e+07,0.0,0
AQ15_2,4.128833e+07,0.0,4.467540e+08,0.0,0.0,0.0,0.0,0.0,6.379603e+07,0.0,...,0.0,0.0,0.0,2.607217e+07,2.000525e+07,0.0,1.996009e+07,1.733512e+07,0.0,0
AQ15_3,4.127786e+07,0.0,4.587334e+08,0.0,0.0,0.0,0.0,0.0,5.962225e+07,0.0,...,0.0,0.0,0.0,2.500974e+07,1.990883e+07,0.0,1.937854e+07,1.604495e+07,0.0,0
AQ24_1,5.993508e+07,0.0,2.692450e+08,0.0,0.0,0.0,0.0,0.0,6.780473e+07,0.0,...,0.0,0.0,0.0,2.357625e+07,1.691154e+07,0.0,1.927812e+07,1.474818e+07,0.0,0
AQ24_2,5.888217e+07,0.0,2.620074e+08,0.0,0.0,0.0,0.0,0.0,6.472963e+07,0.0,...,0.0,0.0,0.0,2.504961e+07,1.758071e+07,0.0,1.803453e+07,1.498274e+07,0.0,0


In [16]:
#maytenus_train.to_csv('features_train_comparison.csv')

Data is now ready for ANY machine learning process

# Machine learning

## X y split

In [17]:
X_train = maytenus_train_grid.drop("class", axis=1)
y_train = maytenus_train_grid["class"]

X_val = maytenus_val.drop("class", axis=1)
y_val = maytenus_val["class"]

## Training

In [18]:
# https://stackoverflow.com/questions/31948879/using-explicit-predefined-validation-set-for-grid-search-with-sklearn
# https://stackoverflow.com/questions/48390601/explicitly-specifying-test-train-sets-in-gridsearchcv

# Create a list of indices for the training and validation sets
train_indices = np.ones(len(X_train))
val_indices = np.zeros(len(X_val))
cv_indices = np.concatenate((train_indices, val_indices))


# model
svm = SVC()
rf = RandomForestClassifier(random_state=2187)
knn = KNeighborsClassifier()

# params of each model

param_svm = {}
param_svm['model'] = [svm]
param_svm['model__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
param_svm['model__C'] = [1, 0.9]
param_svm['model__kernel'] = ['rbf']

param_rf = {}
param_rf['model'] = [rf]
param_rf['model__max_depth'] = [10,15]
param_rf['model__n_estimators'] = [100,200,300]
param_rf['model__criterion'] = ['gini', 'entropy']


param_knn = {}
param_knn['model'] = [knn]
param_knn['model__n_neighbors'] = [5,15,25]


# uniting param to test in gridsearch

params_gridsearch = [param_svm,param_rf,param_knn]

# no need to encode or transform data. All is numeric and same scale

# pipe - starts with svm 
pipe = Pipeline([('model', svm)])

cv = PredefinedSplit(cv_indices)

# gridsearch 
grid = GridSearchCV(pipe, params_gridsearch, 
                    cv = cv,
                   scoring = ['f1','matthews_corrcoef'],
                   return_train_score = True, 
                   refit = 'matthews_corrcoef',
                   verbose = 3)

In [19]:
grid.fit(np.vstack((X_train, X_val)), np.hstack((y_train, y_val)))


Fitting 2 folds for each of 17 candidates, totalling 34 fits
[CV 1/2] END model=SVC(), model__C=1, model__kernel=rbf; f1: (train=0.987, test=1.000) matthews_corrcoef: (train=0.974, test=1.000) total time=   0.0s
[CV 2/2] END model=SVC(), model__C=1, model__kernel=rbf; f1: (train=1.000, test=0.952) matthews_corrcoef: (train=1.000, test=0.908) total time=   0.0s
[CV 1/2] END model=SVC(), model__C=0.9, model__kernel=rbf; f1: (train=0.987, test=1.000) matthews_corrcoef: (train=0.974, test=1.000) total time=   0.0s
[CV 2/2] END model=SVC(), model__C=0.9, model__kernel=rbf; f1: (train=1.000, test=0.950) matthews_corrcoef: (train=1.000, test=0.906) total time=   0.0s
[CV 1/2] END model=RandomForestClassifier(random_state=2187), model__criterion=gini, model__max_depth=10, model__n_estimators=100; f1: (train=1.000, test=1.000) matthews_corrcoef: (train=1.000, test=1.000) total time=   0.5s
[CV 2/2] END model=RandomForestClassifier(random_state=2187), model__criterion=gini, model__max_depth=10, 

GridSearchCV(cv=PredefinedSplit(test_fold=array([1, 1, ..., 0, 0])),
             estimator=Pipeline(steps=[('model', SVC())]),
             param_grid=[{'model': [SVC()], 'model__C': [1, 0.9],
                          'model__kernel': ['rbf']},
                         {'model': [RandomForestClassifier(random_state=2187)],
                          'model__criterion': ['gini', 'entropy'],
                          'model__max_depth': [10, 15],
                          'model__n_estimators': [100, 200, 300]},
                         {'model': [KNeighborsClassifier()],
                          'model__n_neighbors': [5, 15, 25]}],
             refit='matthews_corrcoef', return_train_score=True,
             scoring=['f1', 'matthews_corrcoef'], verbose=3)

In [20]:
pd.DataFrame(grid.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_model', 'param_model__C', 'param_model__kernel',
       'param_model__criterion', 'param_model__max_depth',
       'param_model__n_estimators', 'param_model__n_neighbors', 'params',
       'split0_test_f1', 'split1_test_f1', 'mean_test_f1', 'std_test_f1',
       'rank_test_f1', 'split0_train_f1', 'split1_train_f1', 'mean_train_f1',
       'std_train_f1', 'split0_test_matthews_corrcoef',
       'split1_test_matthews_corrcoef', 'mean_test_matthews_corrcoef',
       'std_test_matthews_corrcoef', 'rank_test_matthews_corrcoef',
       'split0_train_matthews_corrcoef', 'split1_train_matthews_corrcoef',
       'mean_train_matthews_corrcoef', 'std_train_matthews_corrcoef'],
      dtype='object')

In [21]:
grid.best_params_

{'model': KNeighborsClassifier(), 'model__n_neighbors': 5}

In [22]:
grid.best_score_

0.9626932234861232

In [23]:
round(grid.best_score_,4)

0.9627

In [24]:
results = pd.DataFrame(grid.cv_results_)

In [39]:
(results[['param_model','mean_train_matthews_corrcoef','mean_test_matthews_corrcoef']]
         .groupby('param_model',sort=False)
         .mean(['mean_train_matthews_corrcoef','mean_test_matthews_corrcoef'])
         .round(3))

,mean_train_matthews_corrcoef,mean_test_matthews_corrcoef
param_model,,
SVC(),0.987,0.953
RandomForestClassifier(random_state=2187),1.000,0.950
KNeighborsClassifier(),0.986,0.963


In [40]:
(results[['param_model','mean_train_f1','mean_test_f1']]
         .groupby('param_model',sort=False)
         .mean(['mean_train_f1','mean_test_f1'])
         .round(3))

,mean_train_f1,mean_test_f1
param_model,,
SVC(),0.994,0.975
RandomForestClassifier(random_state=2187),1.000,0.974
KNeighborsClassifier(),0.993,0.981


# Scores to present

In [38]:
results[['param_model','mean_test_f1','std_test_f1','mean_test_matthews_corrcoef','std_test_matthews_corrcoef']].groupby('param_model',sort=False).mean(['mean_test_f1','std_test_f1','mean_test_matthews_corrcoef','std_test_matthews_corrcoef']).round(3)

,mean_test_f1,std_test_f1,mean_test_matthews_corrcoef,std_test_matthews_corrcoef
param_model,,,,
SVC(),0.975,0.025,0.953,0.047
RandomForestClassifier(random_state=2187),0.974,0.020,0.950,0.038
KNeighborsClassifier(),0.981,0.019,0.963,0.037
